In [1]:
#!pip install plotly_express
#!pip install pandas 

In [56]:
# import libraries
import pandas as pd 
import plotly_express as px 
import json

In [3]:
# first dataset
try:
    url1 = 'datasets/tinder.csv'
    tinder = pd.read_csv(url1, on_bad_lines='skip')
    print('local')
except:
    print('remote')
    url1 = 'https://raw.githubusercontent.com/dataPracticum/code_pudding_v2/main/Tinder.csv'
    tinder = pd.read_csv(url1, on_bad_lines='skip', names=['id', 'usage', 'university', 'answer', 'matches', 'percentage', 'relationship'])

local


In [4]:
tinder.head()

,id,usage,university,answer,matches,percentage,relationship
0,292890.8970,web,"Meridian, Idaho",No,0.000000,0.000000,0
1,292887.9870,web,"Meridian, Idaho",No,0.000000,0.000000,0
2,292894.0656,gender,"Meridian, Idaho",No,499.173606,0.225255,0
3,292887.1180,web,"Meridian, Idaho",No,0.000000,0.000000,0
4,292893.6561,gender,"Meridian, Idaho",No,455.925963,0.211360,0


In [57]:
# function to convert to json
def convert_json(df):
    df_json = df.to_json()
    data = json.loads(df_json)
    new_df = json.dumps(data, indent=2)
    return print(new_df)

In [58]:
convert_json(tinder)

{
  "id": {
    "0": 292890.897,
    "1": 292887.987,
    "2": 292894.0656,
    "3": 292887.118,
    "4": 292893.6561,
    "5": 292896.2932,
    "6": 292889.6679,
    "7": 293223.0,
    "8": 292881.1384,
    "9": 292889.6084,
    "10": 292890.3917,
    "11": 292901.5235,
    "12": 292892.2684,
    "13": 292887.4288,
    "14": 292897.9815,
    "15": 292884.2696,
    "16": 292902.2716,
    "17": 292889.3406,
    "18": 293009.0,
    "19": 292943.0117,
    "20": 292887.016,
    "21": 293099.0,
    "22": 292882.5488,
    "23": 292883.4428,
    "24": 292897.8191,
    "25": 292899.4996,
    "26": 292890.8158,
    "27": 292896.9177,
    "28": 292889.5282,
    "29": 292899.5747,
    "30": 293163.0,
    "31": 293019.1113,
    "32": 292900.0206,
    "33": 293655.4053,
    "34": 292887.1403,
    "35": 293575.0,
    "36": 292902.8277,
    "37": 293272.2777,
    "38": 292902.0408,
    "39": 292886.4858,
    "40": 293045.0,
    "41": 292900.8929,
    "42": 292890.6365,
    "43": 293413.0,
    "44": 2

# Introduction

## Tinder

In [6]:
# look at data
tinder.head()

,id,usage,university,answer,matches,percentage,relationship
0,292890.8970,web,"Meridian, Idaho",No,0.000000,0.000000,0
1,292887.9870,web,"Meridian, Idaho",No,0.000000,0.000000,0
2,292894.0656,gender,"Meridian, Idaho",No,499.173606,0.225255,0
3,292887.1180,web,"Meridian, Idaho",No,0.000000,0.000000,0
4,292893.6561,gender,"Meridian, Idaho",No,455.925963,0.211360,0


In [7]:
# missing values
tinder.isna().sum()

id              0
usage           0
university      0
answer          0
matches         0
percentage      0
relationship    0
dtype: int64

In [8]:
# looking for duplicates
tinder[tinder.duplicated()]

,id,usage,university,answer,matches,percentage,relationship
1295,292889.9062,web,"Meridian, Idaho",No,0.0,0.0,0
1636,292887.7623,web,"Meridian, Idaho",No,0.0,0.0,0


Different id's so they are not actually duplicates

In [9]:
# values of answer
tinder.answer.value_counts()

No                    1571
Yes                    164
I don't use Tinder     161
Name: answer, dtype: int64

In [10]:
# tinder columns
tinder.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1896 entries, 0 to 1895
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            1896 non-null   float64
 1   usage         1896 non-null   object 
 2   university    1896 non-null   object 
 3   answer        1896 non-null   object 
 4   matches       1896 non-null   float64
 5   percentage    1896 non-null   float64
 6   relationship  1896 non-null   int64  
dtypes: float64(3), int64(1), object(3)
memory usage: 103.8+ KB


Tinder is a relatively small dataset with 1,896 values. Target value is relationship, where it determines whether the individual found a relationship using the app. We can use this data to run a ML model to predict wether an individual will get a relationship from Tinder. 

### EDA

In [11]:
# correlation
tinder.corr()

,id,matches,percentage,relationship
id,1.000000,-0.429790,0.164302,0.167766
matches,-0.429790,1.000000,0.412200,0.186139
percentage,0.164302,0.412200,1.000000,0.311429
relationship,0.167766,0.186139,0.311429,1.000000


In [12]:
# summary statistics
tinder.describe()

,id,matches,percentage,relationship
count,1896.000000,1896.000000,1896.000000,1896.000000
mean,292999.394750,446.881267,0.254098,0.350738
std,227.809806,494.110578,0.220809,0.477327
min,292881.000000,0.000000,0.000000,0.000000
25%,292888.196925,0.000000,0.000000,0.000000
50%,292893.904400,400.439633,0.257873,0.000000
75%,292902.568750,845.560798,0.396787,1.000000
max,293785.000000,1969.000000,1.000000,1.000000


In [13]:
# Skew
tinder.skew()

C:\Users\XIX\AppData\Local\Temp\ipykernel_26820\3200751176.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  tinder.skew()


id              2.044382
matches         0.716367
percentage      0.816940
relationship    0.626066
dtype: float64

In [14]:
# answers to question: do you use tinder?
tinder.answer.value_counts()

No                    1571
Yes                    164
I don't use Tinder     161
Name: answer, dtype: int64

In [15]:
# compile no answers
tinder.answer.replace('I don\'t use Tinder', 'No', inplace=True)

In [16]:
# answers to question: do you use tinder?
tinder.answer.value_counts()

No     1732
Yes     164
Name: answer, dtype: int64

In [17]:
# use tinder and found relationship
tinder[tinder.answer=='Yes'].relationship.value_counts()

1    86
0    78
Name: relationship, dtype: int64

In [18]:
# relationship found using tinder 
print(86/(86+78) * 100, '%')

52.4390243902439 %


In [19]:
# dont use tinder
tinder[tinder.answer=='No'].relationship.value_counts()

0    1153
1     579
Name: relationship, dtype: int64

In [20]:
# relationship found not using tinder
print(579/(1153+579) * 100, '%')

33.429561200923786 %


In [21]:
# dataframe comparing relationship success
data = {'category': ['Tinder', 'Not Tinder'], 'percentage':[52.44, 33.43]}
comparison = pd.DataFrame(data)

In [62]:
# convert_json(comparison)

{
  "category": {
    "0": "Tinder",
    "1": "Not Tinder"
  },
  "percentage": {
    "0": 52.44,
    "1": 33.43
  }
}


##### Bar 1

In [22]:
px.bar(comparison, x='category', y='percentage', color='category', title='Relationship Success Rates')

In [60]:
convert_json(px.bar(comparison, x='category', y='percentage', color='category', title='Relationship Success Rates'))

{
  "data": [
    {
      "alignmentgroup": "True",
      "hovertemplate": "category=%{x}<br>percentage=%{y}<extra></extra>",
      "legendgroup": "Tinder",
      "marker": {
        "color": "#636efa",
        "pattern": {
          "shape": ""
        }
      },
      "name": "Tinder",
      "offsetgroup": "Tinder",
      "orientation": "v",
      "showlegend": true,
      "textposition": "auto",
      "x": [
        "Tinder"
      ],
      "xaxis": "x",
      "y": [
        52.44
      ],
      "yaxis": "y",
      "type": "bar"
    },
    {
      "alignmentgroup": "True",
      "hovertemplate": "category=%{x}<br>percentage=%{y}<extra></extra>",
      "legendgroup": "Not Tinder",
      "marker": {
        "color": "#EF553B",
        "pattern": {
          "shape": ""
        }
      },
      "name": "Not Tinder",
      "offsetgroup": "Not Tinder",
      "orientation": "v",
      "showlegend": true,
      "textposition": "auto",
      "x": [
        "Not Tinder"
      ],
      "xaxis"

In [23]:
# university names
tinder.university.value_counts()

Meridian, Idaho                       1421
Westport, CT                            14
University of Pennsylvania              12
University of Mississippi               10
University of Washington                10
                                      ... 
Boston College                           1
Southern Methodist University            1
Mississippi State University             1
Washington University in St. Louis       1
North Carolina State                     1
Name: university, Length: 148, dtype: int64

In [24]:
# medium of usage
tinder.usage.value_counts()

gender        488
mobile        483
university    464
web           461
Name: usage, dtype: int64

In [25]:
# target values
tinder.relationship.value_counts()

0    1231
1     665
Name: relationship, dtype: int64

##### Fig 2

In [65]:
convert_json(tinder.answer.value_counts())

{
  "No": 1732,
  "Yes": 164
}


In [26]:
px.pie(tinder.answer, names='answer', title='Total Percentage of Tinder Users', template='plotly_white', height=600)

In [63]:
# convert_json(px.pie(tinder.answer, names='answer', title='Total Percentage of Tinder Users', template='plotly_white', height=600))

{
  "data": [
    {
      "domain": {
        "x": [
          0.0,
          1.0
        ],
        "y": [
          0.0,
          1.0
        ]
      },
      "hovertemplate": "answer=%{label}<extra></extra>",
      "labels": [
        "No",
        "No",
        "No",
        "No",
        "No",
        "No",
        "No",
        "Yes",
        "No",
        "No",
        "No",
        "No",
        "No",
        "No",
        "No",
        "No",
        "No",
        "No",
        "No",
        "Yes",
        "No",
        "No",
        "No",
        "No",
        "No",
        "No",
        "No",
        "No",
        "No",
        "No",
        "Yes",
        "No",
        "No",
        "No",
        "No",
        "No",
        "No",
        "Yes",
        "No",
        "No",
        "No",
        "No",
        "No",
        "No",
        "No",
        "Yes",
        "No",
        "Yes",
        "No",
        "No",
        "No",
        "No",
        "No",
        "No",
       

More people do not use Tinder

In [66]:
convert_json(tinder.relationship.value_counts())

{
  "0": 1231,
  "1": 665
}


In [27]:
px.pie(, names='relationship', title='Total Percentage of Relationships', template='plotly_white', height=600)tinder.relationship

More people are not ina relationship

In [28]:
px.pie(tinder[tinder.answer=='Yes'].relationship, names='relationship', title='Relationships Found Using Tinder', template='plotly_white', height=600)

A little more than half of the people that use tinder have found a relationship using the app

In [29]:
px.pie(tinder[tinder.answer=='No'].relationship, names='relationship', title='Found a Relationship Without Tinder', template='plotly_white', height=600,)


A third of People not using Tinder have found a relationship

In [30]:
columns = ['usage', 'answer', 'matches', 'percentage',
       'relationship']
for column in columns:
    px.histogram(tinder[column], title='Distribution of '+ str.upper(column), labels={'value': str.upper(column)}, template='plotly_white', color_discrete_sequence=['red']).show()

In [31]:
# number of unique universities
tinder.university.nunique()

148

In [32]:
tinder.university.value_counts().nlargest(20)

Meridian, Idaho                         1421
Westport, CT                              14
University of Pennsylvania                12
University of Mississippi                 10
University of Washington                  10
University of Southern California          9
Washington State University                8
University of Tampa                        8
Wesleyan University                        8
University of Wisconsin, Eau Claire        8
University of California, Santa Cruz       8
University of California, Berkeley         8
Williams College                           7
Vassar College                             7
Vanderbilt University                      7
University of Buffalo                      7
University of Oregon                       7
Union College                              7
Whatsgoodly University                     6
University of Delaware                     6
Name: university, dtype: int64

In [33]:
px.bar(tinder.university.value_counts().nlargest(20), title='Distribution of Most Common Universities', labels={'value': 'Universities'}, template='plotly_white', color_discrete_sequence=['red']).show()

Most common university is Meridian in Ohio. The other universities make up a small percentage, roughly 15%  of the rest of the schools.

In [34]:
columns = ['matches', 'percentage']
for column in columns:
    px.box(tinder[column], title='Distribution of '+ str.upper(column), labels={'value': str.upper(column)}, template='plotly_white', color_discrete_sequence=['red']).show()

Average matches are around 400, with the max at 1969. The minimm amount of matches is 0, while the upper quartile is 846. Most people get matches on .25% of swipes, while the interquartile range is 0-0.4%.  

In [35]:
px.imshow(tinder.corr(), text_auto=True, title='Tinder Correlations', template='plotly_white', aspect='equal', height=600, color_continuous_scale='hot')

# Looking for Love in All the Wrong Places?

* Are you looking for love? What are your chances of finding it on tinder, and what are your chances of finding it by yourself? 
* Can we predict your chances on tinder based on your school and average stats?


## ML Predicting Relationship

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder

In [37]:
encoder = OrdinalEncoder()
data_ordinal = pd.DataFrame(encoder.fit_transform(tinder), columns=tinder.columns)

X = data_ordinal.drop(columns=['relationship', 'id'])
y = data_ordinal.relationship



In [38]:
X.head()

,usage,university,answer,matches,percentage
0,3.0,46.0,0.0,0.0,0.0
1,3.0,46.0,0.0,0.0,0.0
2,0.0,46.0,0.0,275.0,157.0
3,3.0,46.0,0.0,0.0,0.0
4,0.0,46.0,0.0,201.0,100.0


In [39]:
y.value_counts()

0.0    1231
1.0     665
Name: relationship, dtype: int64

#### Usage Map

In [40]:
# show  map of coded values
tinder.usage.value_counts()

gender        488
mobile        483
university    464
web           461
Name: usage, dtype: int64

In [41]:
# label encoded values
X.usage.value_counts()

0.0    488
1.0    483
2.0    464
3.0    461
Name: usage, dtype: int64

#### University Map

In [42]:
# show  map of coded values
tinder.university.value_counts().nlargest(20)

Meridian, Idaho                         1421
Westport, CT                              14
University of Pennsylvania                12
University of Mississippi                 10
University of Washington                  10
University of Southern California          9
Washington State University                8
University of Tampa                        8
Wesleyan University                        8
University of Wisconsin, Eau Claire        8
University of California, Santa Cruz       8
University of California, Berkeley         8
Williams College                           7
Vassar College                             7
Vanderbilt University                      7
University of Buffalo                      7
University of Oregon                       7
Union College                              7
Whatsgoodly University                     6
University of Delaware                     6
Name: university, dtype: int64

In [43]:
# coded values
X.university.value_counts().nlargest(20)

46.0     1421
144.0      14
123.0      12
117.0      10
133.0      10
127.0       9
140.0       8
129.0       8
143.0       8
134.0       8
97.0        8
92.0        8
146.0       7
139.0       7
138.0       7
91.0        7
122.0       7
86.0        7
145.0       6
104.0       6
Name: university, dtype: int64

May be easier to assume Meridian, as it is the most prevalent, or at least one of the top 20 universities.

In [44]:
# new data to make predictions
new_data = pd.DataFrame(index=['usage', 'university', 'answer', 'matches', 'percentage'], data=[3, 46, 1, 400, 0.25])
new_data = new_data.T

In [45]:
new_data

,usage,university,answer,matches,percentage
0,3.0,46.0,1.0,400.0,0.25


In [46]:
# Classifier pipeline
pipe_lr = Pipeline([('scalar1', StandardScaler()), ('lr_classifier', LogisticRegression(random_state=19, max_iter=1000))])
pipe_dt = Pipeline([('scalar1', StandardScaler()), ('dt_classifier', DecisionTreeClassifier(random_state=19))])
pipe_rf = Pipeline([('scalar1', StandardScaler()), ('rf_classifier', RandomForestClassifier(random_state=19))])
pipe_sv = Pipeline([('scalar1', StandardScaler()), ('svm_classifier', svm.SVC(random_state=19))])
pipe_nb = Pipeline([('scalar1', StandardScaler()), ('nb_classifier', GaussianNB())])
pipe_kn = Pipeline([('scalar1', StandardScaler()), ('knn_classifier', KNeighborsClassifier(n_neighbors=3))])

pipelines = [pipe_lr, pipe_dt, pipe_rf, pipe_sv, pipe_nb, pipe_kn]

best_accuracy = 0
best_classifier = 0
best_pipeline = ""

pipe_dict = {0: 'Logistic Regression', 1: 'Decision Tree', 2: 'Random Forest', 3: 'SVM', 4: 'Naive-Bayes', 5: 'KNN'}

# Use cross-validation to evaluate the models
for i, model in enumerate(pipelines):
    scores = cross_val_score(model, X, y, cv=5, scoring='f1')
    print('{} Cross-Validation Accuracy: {:.2f}'.format(pipe_dict[i], scores.mean()))
    if scores.mean() > best_accuracy:
        best_accuracy = scores.mean()
        best_pipeline = model
        best_classifier = i

# Print the best classifier
print('\nClassifier with the best accuracy: {}'.format(pipe_dict[best_classifier]))

Logistic Regression Cross-Validation Accuracy: 0.41
Decision Tree Cross-Validation Accuracy: 0.83
Random Forest Cross-Validation Accuracy: 0.85
SVM Cross-Validation Accuracy: 0.83
Naive-Bayes Cross-Validation Accuracy: 0.52
KNN Cross-Validation Accuracy: 0.85

Classifier with the best accuracy: Random Forest


In [47]:
# XGB

import xgboost as xgb

X2 = tinder.drop(columns=['relationship', 'id'])
y2 = tinder.relationship

lbl = LabelEncoder()
X2.usage = lbl.fit_transform(X2.usage.astype(str))
X2.university = lbl.fit_transform(X2.university.astype(str))
X2.answer = lbl.fit_transform(X2.answer.astype(str))
X2.matches = lbl.fit_transform(X2.matches.astype(str))
X2.percentage = lbl.fit_transform(X2.percentage.astype(str))



X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, y2, test_size=0.25, random_state=19)

# Create a XGBM 
xgbc = xgb.XGBClassifier(random_state=19)

xgbc.fit(X2_train, y2_train)

scores = cross_val_score(xgbc, X2, y2, cv=5, scoring='f1') 
final_score = sum(scores) / len(scores)
print('Average model evaluation score:', final_score)

Average model evaluation score: 0.8586813125972839


In [48]:
# new data to make predictions with categorical columns
new_data2 = pd.DataFrame(index=['usage', 'university', 'answer', 'matches', 'percentage'], data=['web','Meridian, Idaho' , 'Yes', 400, 0.25])
new_data2 = new_data2.T

In [49]:
new_data2.matches = new_data2.matches.astype('float')
new_data2.percentage = new_data2.percentage.astype('float')
new_data2.answer = new_data2.answer.astype('category')
new_data2.university = new_data2.university.astype('category')
new_data2.usage = new_data2.usage.astype('category')

In [50]:
new_data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   usage       1 non-null      category
 1   university  1 non-null      category
 2   answer      1 non-null      category
 3   matches     1 non-null      float64 
 4   percentage  1 non-null      float64 
dtypes: category(3), float64(2)
memory usage: 495.0 bytes


In [51]:
xgbc.predict(new_data2)

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:usage: category, university: category, answer: category

In [52]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.2, 0.3],
    'n_estimators': [100, 200, 300],
}

# Create the XGBoost model
xgb = XGBClassifier()

# Create the GridSearchCV object
grid_search = GridSearchCV(xgb, param_grid, cv=5, scoring='f1', verbose=3, n_jobs=-1)


# Fit the GridSearchCV object to the data
grid_search.fit(X, y)

scores = cross_val_score(grid_search.best_estimator_, X, y, cv=5, scoring='f1') 
final_score = sum(scores) / len(scores)
print('Average model evaluation score:', final_score)


Fitting 5 folds for each of 27 candidates, totalling 135 fits
Average model evaluation score: 0.868553821658959


In [53]:
# prediction of relationship status based on new data
grid_search.best_estimator_.predict(new_data)

array([0])

Prediction is single based on the new data

In [54]:
# catboost

from catboost import CatBoostClassifier, Pool


catb = CatBoostClassifier(task_type='GPU', loss_function='Logloss', eval_metric='AUC', iterations=1500, random_seed=19)

catb.fit(X, y, verbose=100, use_best_model=True, plot=True)

scores = cross_val_score(catb, X, y, cv=2, scoring='f1') 
final_score = sum(scores) / len(scores)
print('Average model evaluation score:', final_score)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

You should provide test set for use best model. use_best_model parameter has been switched to false value.
Default metric period is 5 because AUC is/are not implemented for GPU


Learning rate set to 0.022632
0:	total: 56.1ms	remaining: 1m 24s
100:	total: 4.82s	remaining: 1m 6s
200:	total: 9.22s	remaining: 59.6s
300:	total: 13.2s	remaining: 52.7s
400:	total: 17.3s	remaining: 47.5s
500:	total: 21.4s	remaining: 42.6s
600:	total: 25.9s	remaining: 38.7s
700:	total: 30.4s	remaining: 34.7s
800:	total: 34.5s	remaining: 30.1s
900:	total: 38.9s	remaining: 25.8s
1000:	total: 43.3s	remaining: 21.6s
1100:	total: 47.2s	remaining: 17.1s
1200:	total: 51.2s	remaining: 12.7s
1300:	total: 55.3s	remaining: 8.46s
1400:	total: 59s	remaining: 4.17s
1499:	total: 1m 2s	remaining: 0us
Learning rate set to 0.023511


Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 45.6ms	remaining: 1m 8s
1:	total: 93.3ms	remaining: 1m 9s
2:	total: 141ms	remaining: 1m 10s
3:	total: 187ms	remaining: 1m 9s
4:	total: 240ms	remaining: 1m 11s
5:	total: 292ms	remaining: 1m 12s
6:	total: 333ms	remaining: 1m 10s
7:	total: 380ms	remaining: 1m 10s
8:	total: 427ms	remaining: 1m 10s
9:	total: 468ms	remaining: 1m 9s
10:	total: 522ms	remaining: 1m 10s
11:	total: 570ms	remaining: 1m 10s
12:	total: 615ms	remaining: 1m 10s
13:	total: 656ms	remaining: 1m 9s
14:	total: 703ms	remaining: 1m 9s
15:	total: 757ms	remaining: 1m 10s
16:	total: 807ms	remaining: 1m 10s
17:	total: 853ms	remaining: 1m 10s
18:	total: 897ms	remaining: 1m 9s
19:	total: 944ms	remaining: 1m 9s
20:	total: 991ms	remaining: 1m 9s
21:	total: 1.04s	remaining: 1m 9s
22:	total: 1.08s	remaining: 1m 9s
23:	total: 1.12s	remaining: 1m 8s
24:	total: 1.16s	remaining: 1m 8s
25:	total: 1.2s	remaining: 1m 8s
26:	total: 1.24s	remaining: 1m 7s
27:	total: 1.28s	remaining: 1m 7s
28:	total: 1.32s	remaining: 1m 7s
29:	total: 

Default metric period is 5 because AUC is/are not implemented for GPU


4:	total: 244ms	remaining: 1m 12s
5:	total: 292ms	remaining: 1m 12s
6:	total: 337ms	remaining: 1m 11s
7:	total: 387ms	remaining: 1m 12s
8:	total: 434ms	remaining: 1m 11s
9:	total: 482ms	remaining: 1m 11s
10:	total: 532ms	remaining: 1m 12s
11:	total: 575ms	remaining: 1m 11s
12:	total: 618ms	remaining: 1m 10s
13:	total: 661ms	remaining: 1m 10s
14:	total: 711ms	remaining: 1m 10s
15:	total: 757ms	remaining: 1m 10s
16:	total: 799ms	remaining: 1m 9s
17:	total: 841ms	remaining: 1m 9s
18:	total: 888ms	remaining: 1m 9s
19:	total: 932ms	remaining: 1m 8s
20:	total: 968ms	remaining: 1m 8s
21:	total: 1.01s	remaining: 1m 8s
22:	total: 1.06s	remaining: 1m 8s
23:	total: 1.1s	remaining: 1m 7s
24:	total: 1.14s	remaining: 1m 7s
25:	total: 1.19s	remaining: 1m 7s
26:	total: 1.24s	remaining: 1m 7s
27:	total: 1.28s	remaining: 1m 7s
28:	total: 1.33s	remaining: 1m 7s
29:	total: 1.37s	remaining: 1m 7s
30:	total: 1.42s	remaining: 1m 7s
31:	total: 1.46s	remaining: 1m 7s
32:	total: 1.51s	remaining: 1m 6s
33:	total